In [2]:
from __future__ import unicode_literals
from matplotlib import pyplot as plt
import pickle
import pandas as pd
import numpy as np
import nltk as nk
import untangle
import seaborn as sns
import operator
import string 
nk.download('punkt')
nk.download('stopwords')
nk.download('wordnet')
nk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/jlanuza/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jlanuza/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jlanuza/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jlanuza/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Asociación de palabras

# bigramcollocationfinder.from_words

#### 1. Levantar el corpus AP, separando cada noticia como un elemento distinto en un diccionario (DOCNO :TEXT).


In [3]:
corpus_ap = {}
obj = untangle.parse('ap/ap.txt')
for i in range(len(obj.root.DOC)):
    documento = obj.root.DOC[i]
    corpus_ap[documento.DOCNO.cdata.strip()] = documento.TEXT.cdata.strip()

### 2. Calcular el tamaño del vocabulario

#### Pasos de filtrado

1. Tokenizamos oraciones
2. Para cada oración realizamos una tokenización de palabras básica (separamos por espacios)
3. Computamos el POS tag de cada palabra
4. Usando el POS tag lematizamos cada palabra
5. Para cada palabra lematizada aumentamos en 1 la cantidad de veces que aparece

In [52]:
def clean_chars(word):
    replaceable = [",", ".", "-", "_", "`", ":", ";", "!", "?", "`", "'"]
    for token in replaceable:
        word = word.replace(token, " ")
    return word.strip()

stopw = set(nk.corpus.stopwords.words("english"))


documents = []

for (docno, text) in corpus_ap.items():
    tokens = nk.sent_tokenize(text.lower())
    document = ""
    for sent in tokens:
        document += sent
    documents.append(document)

len(documents)
    
words_by_doc = [[clean_chars(w) for w in d.split() if len(clean_chars(w)) > 1 or clean_chars(w).isalpha()] 
                  for d in documents]
        
words_by_doc = [[w for w in sent if w not in stopw and len(w) > 1] for sent in words_by_doc]

In [53]:
word_semantic = nk.tag.pos_tag_sents(words_by_doc)

In [60]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return nk.corpus.wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return nk.corpus.wordnet.VERB
    elif treebank_tag.startswith('N'):
        return nk.corpus.wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return nk.corpus.wordnet.ADV
    else:
        return nk.corpus.wordnet.NOUN

lemmatizer = nk.WordNetLemmatizer()

word_count = {}

clean_texts = []

for document in word_semantic:
    text = ""
    for (word, tag) in document:
        lemmatized = lemmatizer.lemmatize(word, get_wordnet_pos(tag))
        word = lemmatized.split()[0]
        if not word[0].isalpha():
            continue
        elif word not in word_count:
            word_count[word] = 1
        else:
            word_count[word] += 1
        text += " " + word
    clean_texts.append(text)
    
with open('word_count2.pickle', 'wb') as handle:
    pickle.dump(word_count, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [69]:
with open('word_count2.pickle', 'rb') as handle:
    word_count = pickle.load(handle) 

word_count = [(x, y) for (x, y) in sorted(word_count.items(), key=operator.itemgetter(1), reverse=True)]

In [70]:
word_count[:500]

[(u'say', 10730),
 (u'said', 3534),
 (u'year', 2622),
 (u'would', 2561),
 (u'state', 2081),
 (u'new', 1993),
 (u'u', 1957),
 (u'percent', 1942),
 (u'one', 1705),
 (u'people', 1663),
 (u'million', 1630),
 (u'two', 1608),
 (u'government', 1556),
 (u'also', 1535),
 (u'president', 1513),
 (u'make', 1478),
 (u'last', 1373),
 (u'official', 1353),
 (u'take', 1333),
 (u'go', 1325),
 (u'report', 1266),
 (u'time', 1236),
 (u'first', 1144),
 (u'soviet', 1109),
 (u'include', 1056),
 (u'could', 1050),
 (u'day', 1045),
 (u'get', 1005),
 (u'company', 963),
 (u'week', 960),
 (u'three', 956),
 (u'tell', 949),
 (u'american', 941),
 (u'bush', 934),
 (u'use', 926),
 (u'work', 905),
 (u'billion', 894),
 (u'police', 894),
 (u'call', 887),
 (u'today', 878),
 (u'give', 857),
 (u'come', 855),
 (u'country', 845),
 (u'group', 836),
 (u'force', 833),
 (u'plan', 826),
 (u'month', 818),
 (u'national', 818),
 (u'high', 813),
 (u'city', 807),
 (u'court', 803),
 (u'house', 802),
 (u'united', 792),
 (u'thursday', 792),

In [71]:
print "Tamaño del vocabulario: ", len(word_count), " palabras."

Tamaño del vocabulario:  29435  palabras.


#### 3. Para las 500 palabras con más apariciones, calcular el par más asociado según la medida presentada.

In [74]:
most_used = word_count[:500]

Para el cálculo utilizaremos una ventana de 5 palabras como indica el paper.

In [ ]:
def get_window(texto, word):
    print texto

In [97]:
def get_close_words(words, idx):
    return words[max(0, idx-5):idx] + words[idx+1:min(len(words)-1,idx + 6)]

window_size = 5

asociaciones = {}
for word,amt in most_used:
        asociaciones[word] = {}
print len(clean_texts)
i = 0
for document in clean_texts:
    print i,
    i+=1
    for (target, amt) in most_used:
        sdoc = document.split()
        for idx in range(len(sdoc)):
            if sdoc[idx] == target:
                for pareja in get_close_words(sdoc, idx):
                    asociaciones[target][pareja] = \
                        asociaciones[target][pareja] + 1 if pareja in asociaciones[target] else 1

2250


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

 1876 1877 1878 1879 1880 1881 1882 1883 1884 1885 1886 1887 1888 1889 1890 1891 1892 1893 1894 1895 1896 1897 1898 1899 1900 1901 1902 1903 1904 1905 1906 1907 1908 1909 1910 1911 1912 1913 1914 1915 1916 1917 1918 1919 1920 1921 1922 1923 1924 1925 1926 1927 1928 1929 1930 1931 1932 1933 1934 1935 1936 1937 1938 1939 1940 1941 1942 1943 1944 1945 1946 1947 1948 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024 2025 2026 2027 2028 2029 2030 2031 2032 2033 2034 2035 2036 2037 2038 2039 2040 2041 2042 2043 2044 2045 2046 2047 2048 2049 2050 2051 2052 2053 2054 2055 2056 2057 2058 2059 2060 2061 2062 2063 2064 2065 2066 2067 2068 2069 2070 2071 2072 2073 2074 2075

In [118]:
palabras_totales = sum([y for (x, y)  in word_count])

def freq(x):
    return dict(word_count)[x] / float(palabras_totales)

def info_mutua(x, y):       
    if y not in asociaciones[x]:
        return 0
    else:
#         print float(asociaciones[x][y])/palabras_totales, freq(x), freq(y)
        return (float(asociaciones[x][y])/palabras_totales) / (freq(x)*freq(y)) 


print info_mutua('say', 'wednesday')

16.8808353808


In [124]:
maximas_relacionadas = {}
for palabra_estrella, cant in most_used[0:1]:
    maximas_relacionadas[palabra_estrella] = ('', 0)
    palabras_relacionadas = asociaciones[palabra_estrella].keys()
    print len(palabras_relacionadas)
    for palabra_relacionada in palabras_relacionadas:
        i_mutua = info_mutua(palabra_estrella, palabra_relacionada)
        if maximas_relacionadas[palabra_estrella][1] < i_mutua:
            maximas_relacionadas[palabra_estrella] = (palabra_relacionada, i_mutua)
maximas_relacionadas        

14311


{u'say': (u'fairield', 143.24594594594592)}

## Información Léxica[2]
Bajar de Project Gutenberg el libro de Darwin ON THE ORIGIN OF SPECIES.

#### 1. Procesar el texto, tokenizando eliminando signos de puntuación.

In [125]:
tokenizer = nk.tokenize.RegexpTokenizer(r'\w+')
with open('darwin.txt', 'r') as darwintexto:
    data= tokenizer.tokenize(darwintexto.read())

In [128]:
data = [d.lower() for d in data]
data

['on',
 'the',
 'origin',
 'of',
 'species',
 'introduction',
 'when',
 'on',
 'board',
 'h',
 'm',
 's',
 'beagle',
 'as',
 'naturalist',
 'i',
 'was',
 'much',
 'struck',
 'with',
 'certain',
 'facts',
 'in',
 'the',
 'distribution',
 'of',
 'the',
 'inhabitants',
 'of',
 'south',
 'america',
 'and',
 'in',
 'the',
 'geological',
 'relations',
 'of',
 'the',
 'present',
 'to',
 'the',
 'past',
 'inhabitants',
 'of',
 'that',
 'continent',
 'these',
 'facts',
 'seemed',
 'to',
 'me',
 'to',
 'throw',
 'some',
 'light',
 'on',
 'the',
 'origin',
 'of',
 'species',
 'that',
 'mystery',
 'of',
 'mysteries',
 'as',
 'it',
 'has',
 'been',
 'called',
 'by',
 'one',
 'of',
 'our',
 'greatest',
 'philosophers',
 'on',
 'my',
 'return',
 'home',
 'it',
 'occurred',
 'to',
 'me',
 'in',
 '1837',
 'that',
 'something',
 'might',
 'perhaps',
 'be',
 'made',
 'out',
 'on',
 'this',
 'question',
 'by',
 'patiently',
 'accumulating',
 'and',
 'reflecting',
 'on',
 'all',
 'sorts',
 'of',
 'facts',


#### 2. Siguiendo el artículo de la sección, calcular la autocorrelación para estimar la distribución de la palabra a lo largo del texto.

In [ ]:
laspalabra = ['for', 'plants', 'instinct']
for lapalabra in laspalabra:

#### 3.  Calcular la entrop´ıa de una selecci´on de 100 palabras que abarquen el rango de frecuencia de aparici´on en el libro, es decir elegir palabras que son muy frecuentes y otras de baja frecuencia.

#### 4. Calcular la entrop´ıa de las palabras seleccionadas anteriormente, randomizando su posici´on en el texto. Comparar con los resultados del punto anterior.

## Word embeddings, distancia semántica y WordNet

#### 1. Utilizando el test WordSim3531, comparar el rendimiento entre LSA[3] y Word2Vec2

#### 2. Comparar los distintos word embeddings con las medidas definidas en WordNet.